In [6]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

data = pd.read_csv('student_enrollment_data.csv')

# Inspect the data (optional)
print(data.head())
print(data.info())

# Define features and target variables
X = data[['GPA', 'attendance_rate', 'test_scores', 'socioeconomic_status']]
y_enroll = data['will_enroll']
y_support = data['needs_support']
X_train, X_test, y_train, y_test = train_test_split(X, y_enroll, test_size=0.2, random_state=42)

# Data Preprocessing: Scaling the features to normalize them
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model Initialization: Random Forest Classifier
# Using Random Forest as it works well with both numerical and categorical features
rf_model = RandomForestClassifier(random_state=42)

# Hyperparameter tuning using GridSearchCV to find the best parameters
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up GridSearchCV with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Select the best model
best_rf_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Train the best model on the training data
best_rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = best_rf_model.predict(X_test)

# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print evaluation results
print("Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"ROC-AUC Score: {roc_auc:.2f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Feature Importance Analysis
# This will help in interpreting which features contribute most to the predictions
feature_importances = best_rf_model.feature_importances_
for feature, importance in zip(X.columns, feature_importances):
    print(f"{feature}: {importance:.4f}")

# Optional: Save the model for future use
import joblib
joblib.dump(best_rf_model, 'student_enrollment_model.pkl')


   student_id  GPA  attendance_rate  test_scores  socioeconomic_status  \
0           1  3.2             0.95           88                     3   
1           2  2.5             0.80           75                     2   
2           3  3.8             0.90           92                     4   
3           4  2.9             0.70           68                     1   
4           5  3.5             0.85           89                     3   

   will_enroll  needs_support  
0            1              0  
1            0              1  
2            1              0  
3            0              1  
4            1              0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   student_id            10 non-null     int64  
 1   GPA                   10 non-null     float64
 2   attendance_rate       10 non-null     float64
 3   te

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.